In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema
from scipy.interpolate import interp1d
from classy import Class

In [ ]:
z_pk = [200.0,100.0,75.0,50.0,10.0,5.0,0.0]
k_out = [0.0001,0.001, 0.01, 0.1,1]
#k_out = [0.0001,0.001]
cosmo500 = Class()
cosmo500.set({'output':'dTk, lTk', 
           'z_pk':str(z_pk).strip('[]'),
           'k_output_values':str(k_out).strip('[]'),
           'a_init_nbody': 1./501.,
              'radiation_streaming_approximation':3,
           'l_max_g':100,
           'l_max_ur':100,
           'reio_parametrization':'reio_none',
           'k_per_decade_for_pk':2,
           'k_per_decade_for_bao':2,
           'gauge' : 'Newtonian',
           'P_k_max_h/Mpc' : 1.,
           'evolver':1,
           'tol_perturb_integration':1e-6,
           'perturb_sampling_stepsize':0.01,
              'tol_background_integration':1e-6,
              'back_integration_stepsize':7e-4,
           'tight_coupling_trigger_tau_c_over_tau_h':0.0015,
           'tight_coupling_trigger_tau_c_over_tau_k':0.0001,
            #'start_sources_at_tau_c_over_tau_h':0.0008
            })
cosmo500.compute()
pts = cosmo500.get_perturbations()['scalar']

Npt = len(pts)
f, axes = plt.subplots(Npt,4,sharex=True,figsize=(14,14))

for j in range(Npt):
    pt = pts[j]
    ax = axes[j]
    tau = pt['tau [Mpc]']
    
    HCtheta = pt['HCtheta']
    HCtheta_prime = pt['HCtheta_prime']
    
    HCtheta_prime_est = np.diff(HCtheta)/np.diff(tau)
    
    #HCtheta_spl = UnivariateSpline(tau,HCtheta,s=0,k=1)
    #HCtheta_prime_spl = HCtheta_spl.derivative()

    HCAnb = pt['HCAnb']
    HCAnb_prime = pt['HCAnb_prime']
    #HCAnb_spl = UnivariateSpline(tau,HCAnb,s=0,k=1)
    #HCAnb_prime_spl = HCAnb_spl.derivative()
    HCAnb_prime_est = np.diff(HCAnb)/np.diff(tau)

    ax[0].semilogy(tau, np.abs(HCtheta))
    ax[2].semilogy(tau, np.abs(HCAnb),ls='--')
    ax[1].semilogy(tau, np.abs(HCtheta_prime))
    ax[1].semilogy(tau[1:], np.abs(HCtheta_prime_est),ls='--',lw=2)
    ax[3].semilogy(tau, np.abs(HCAnb_prime))
    ax[3].semilogy(tau[1:], np.abs(HCAnb_prime_est),ls='--',lw=2)
    
    ax[0].set_ylabel(r'$\mathcal{H}\theta_p$',fontsize=20)
    ax[1].set_ylabel(r'$\frac{d}{d\eta}\left(\mathcal{H}\theta_p\right)$',fontsize=20)
    ax[2].set_ylabel(r'$\mathcal{H}\xi$',fontsize=20)
    ax[3].set_ylabel(r'$\frac{d}{d\eta}\left(\mathcal{H}\xi\right)$',fontsize=20)
    
    ax[0].locator_params(axis='x',nbins=7)

for a in axes[-1]:
    a.set_xlabel(r'$\eta \quad [Mpc]$',fontsize=20)
f.tight_layout()
f.savefig('derivatives.pdf')